In [1]:
from project_lib import Project
project = Project(spark.sparkContext, 'adcc95ed-0e6f-494f-a3a1-d5e74e038455', 'p-0e7c11d93da5347766da7d0e0924866399a71d41')
pc = project.project_context

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200615132334-0003
KERNEL_ID = 7f125647-852b-46ad-b1ef-cd57b7c4c0db


In [9]:
import ibmos2spark
import nltk
!pip install numpy
!pip install gensim 
from gensim.parsing.preprocessing import STOPWORDS

  Using cached https://files.pythonhosted.org/packages/b3/a9/b1bc4c935ed063766bce7d3e8c7b20bd52e515ff1c732b02caacf7918e5a/numpy-1.18.5-cp36-cp36m-manylinux1_x86_64.whl
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
Target directory /home/spark/shared/user-libs/python3.6/numpy-1.18.5.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3.6/numpy already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3.6/numpy.libs already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3.6/bin already exists. Specify --upgrade to force replacement.
  Using cached https://files.pythonhosted.org/packages/2b/e0/fa6326251692056dc880a64eb22117e03269906ba55a6864864d24ec8b4e/gensim-3.8.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/dc/29/162476fd44203116e7980cfbd9352ee

ModuleNotFoundError: No module named 'numpy.version'

In [3]:
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-283f9a60-2385-40f6-973d-6cd059b4731f',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': '8-VH-6DHOLwKlgloUuzelkhXgt0elMaOQxDZB1EznoCu'
}

configuration_name = 'os_c42173278f6a4c65aaac64664b8be689_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_1 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('data.csv', 'bdaproject-donotdelete-pr-qh7hn4gnoctssn'))

In [4]:
import nltk
alphabets = []
tokenizer = nltk.RegexpTokenizer(r"\w+")
for i in range(97,123):
    alphabets.append(chr(i))
def cleaning(line):
    rem_garbage = line.replace("NEWLINE_TOKEN",'').replace("_"," ").replace("1"," ").replace("2"," ").replace("3"," ").replace("4"," ").replace("5"," ").replace("6"," ").replace("7"," ").replace("8"," ").replace("9"," ").replace("0"," ").lower()
    rem_punct = tokenizer.tokenize(rem_garbage)
    rem_stopwords = [word for word in rem_punct if not word in STOPWORDS.union(set(alphabets)) or not word.isalpha()]
    return rem_stopwords

In [5]:
def create_pair(word):
    return (word,1)

In [6]:
coments = df_data_1.select("comment").rdd.collect()

In [7]:
clean_coments = []
for i in range(len(coments)):
    clean_coments.extend(cleaning(coments[i][0]))

NameError: name 'STOPWORDS' is not defined

In [ ]:
clean_coments[:10]

In [ ]:
cleaned_comments = sc.parallelize(clean_coments)
paired_words = cleaned_comments.map(create_pair)

In [ ]:
def sum_counts(a, b):
    return a + b

In [ ]:
corpus_word_count = paired_words.reduceByKey(sum_counts).collect()
new_df = sqlContext.createDataFrame(corpus_word_count,["Words","Count"]).toPandas()
new_df.head()

In [ ]:
!pip install textblob
sentiment = []
from textblob import TextBlob
for i in range(len(new_df)):
    blob = TextBlob(new_df["Words"][i])
    sentiment.append((new_df["Words"][i],new_df["Count"][i],blob.sentiment.polarity))

In [ ]:
import pandas as pd
DF = pd.DataFrame(sentiment,columns=["Words","Count","Sentiment_Score"])


In [ ]:
WCS_DF = DF[DF["Count"]>20].sort_values(by="Count",ascending = False)
WCS_DF.index = range(len(WCS_DF["Words"]))

In [ ]:
rev_ids = df_data_1.select(df_data_1[0]).rdd.collect()
clean_coments = []
for i in range(len(coments)):
    clean_coments.append(cleaning(coments[i][0]))
corpus = []
for j in range(len(clean_coments)):
     corpus.append(' '.join([str(elem) for elem in clean_coments[j]]))
vocab = list(WCS_DF["Words"])
from sklearn.feature_extraction.text import CountVectorizer
df = pd.DataFrame(data=corpus, columns=['reviews'])
vectorizer = CountVectorizer(vocabulary=vocab, min_df=0)
X = vectorizer.fit_transform(df['reviews'].values)
result = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names())

In [ ]:
DF.head(100)

In [ ]:
a = result.tail(1)

In [ ]:
a.reset_index(inplace=True)

In [ ]:
sen = ""
for i in a.columns:
    if a[i][0] == 1:
        print(str(i))
        sen.join(str(i)+" ")

mean
question
came
story
details
worth
clean
dare
brother
alleged
wife
ryan
affair
investigating


In [ ]:
sen = ":There ought to be a dry-break mechanism in the fuel boom to prevent any chance of a fire, let alone an explosion.  It's Hollywood, do you expect realism when you can have drama?  Call it cinematic licence."
aaa = TextBlob(sen)
print(aaa.sentiment.polarity)

0.0


In [ ]:
my_file = project.get_file("label_Score_data.csv")

# Read the CSV data file from the object storage into a pandas DataFrame
my_file.seek(0)
import pandas as pd
data_1 = pd.read_csv(my_file)

In [ ]:
data_1.head()

,rev_id,sentiment,sentiment_score
0,325467263,negative,-0.723434
1,325489890,negative,-0.973351
2,325491573,negative,-0.951228
3,325491602,negative,-0.909637
4,325493177,negative,-0.861186


In [ ]:
my_file = project.get_file("data.csv")

# Read the CSV data file from the object storage into a pandas DataFrame
my_file.seek(0)
data = pd.read_csv(my_file)

In [ ]:
data.head()

,rev_id,comment,year,ns,logged_in,sample,Aggressive_Score,Attack_Score,Toxicity_Score
0,325467263,NEWLINE_TOKEN:There ought to be a dry-break me...,2009,article,True,blocked,0.000000,0.000000,0.0
1,325489890,NEWLINE_TOKENNEWLINE_TOKEN==I AM NOT VANDALIZI...,2009,user,True,blocked,0.555555,0.333333,0.5
2,325491573,`NEWLINE_TOKENNEWLINE_TOKEN==NO ONE CAN APPARE...,2009,article,True,blocked,0.666666,0.666666,0.9
3,325491602,NEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENWikiped...,2009,user,True,blocked,0.333333,0.222222,0.3
4,325493177,NEWLINE_TOKENNEWLINE_TOKEN=Speedy Deletion=NEW...,2009,article,True,blocked,0.000000,0.000000,0.0


In [ ]:
!pip install nltk
!pip install ibm_watson
!pip install cloudant==2.3.1
!pip install watson-developer-cloud==1.5
!pip install --upgrade pixiedust

  Using cached https://files.pythonhosted.org/packages/1a/a1/6d8fdf4a20ffbbf2bd6003dff47a0628b9e6a4b840c421b0dec27da9376e/regex-2020.6.8-cp36-cp36m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f3/76/4697ce203a3d42b2ead61127b35e5fcc26bba9a35c03b32a2bd342a4c869/tqdm-4.46.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b8/a6/d1a816b89aa1e9e96bcb298eb1ee1854f21662ebc6d55ffa3d7b3b50122b/joblib-0.15.1-py3-none-any.whl
Target directory /home/spark/shared/user-libs/python3.6/joblib-0.15.1.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3.6/nltk already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3.6/joblib already exists. Specify --upgrade to force repla

  Using cached https://files.pythonhosted.org/packages/1a/70/1935c770cb3be6e3a8b78ced23d7e0f3b187f5cbfab4749523ed65d7c9b1/requests-2.23.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/89/e3/afebe61c546d18fb1709a61bee788254b40e736cff7271c7de5de2dc4128/idna-2.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/98/99/def511020aa8f663d4a2cfaa38467539e864799289ff354569e339e375b1/certifi-2020.4.5.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e1/e5/df302e8017440f111c11cc41a6b432838672f5a70aa29227bf58149dc72f/urllib3-1.25.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
ibm-cos-sdk-core 2.6.0 has requirement requests<2.23,>=2.18, but you'll have requests 2.23.0 which is incompatible.
Target directory /home/spark/shared/user-libs/python3.6/urllib3 already exists. Specify --upgrad

  Using cached https://files.pythonhosted.org/packages/e1/e5/df302e8017440f111c11cc41a6b432838672f5a70aa29227bf58149dc72f/urllib3-1.25.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/95/95/f657b6e17f00c3f35b5f68b10e46c3a43af353d8856bd57bfcfb1dbb3e92/setuptools-47.1.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f1/c7/72abda280893609e1ddfff90f8064568bd8bcb2c1770a9d5bb5edb2d1fea/cffi-1.14.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ae/e7/d9c3a176ca4b02024debf82342dab36efadfc5776f9c8db077e8f6e71821/pycparser-2.20-py2.py3-none-any.whl
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
pytest-openfiles 0.5.0 has requirement pytest>=4.6, but you'll have pytest 3.10.1 which is incompatible.
pytest-astropy 0.8.

  Using cached https://files.pythonhosted.org/packages/a4/63/eaec2bd025ab48c754b55e8819af0f6a69e2b1e187611dd40cbbe101ee7f/Markdown-3.2.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1a/70/1935c770cb3be6e3a8b78ced23d7e0f3b187f5cbfab4749523ed65d7c9b1/requests-2.23.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/55/6f/c87dffdd88a54dd26a3a9fef1d14b6384a9933c455c54ce3ca7d64a84c88/lxml-4.5.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e4/8d/9e28e9af95739e6d2d2f8d4bef0b3432da40b7c3588fbad4298c1be09e48/geojson-2.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/74/46/e81907704ab203206769dee1385dc77e1407576ff8f50a0681d0a6b541be/colour-0.1.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/pa

In [ ]:
credentials_2 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-ce4a4a44-022d-40fe-be9b-e16e72b4153c',
    'IBM_API_KEY_ID': 'w3-S42mlOXATWHVO_5uU5NiQ2R1EiZxELIupujtqVLIa',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'bdaproject-donotdelete-pr-ku47cnvx97em5d',
    'FILE': 'data.csv'
}

In [ ]:
import json
import watson_developer_cloud
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 \
  import Features, EntitiesOptions, KeywordsOptions
    
import ibm_boto3
from botocore.client import Config

import re
import nltk
from nltk import word_tokenize,sent_tokenize,ne_chunk

cos = ibm_boto3.client('s3',
                    ibm_api_key_id=credentials_2['IBM_API_KEY_ID'],
                    ibm_service_instance_id=credentials_2['IAM_SERVICE_ID'],
                    ibm_auth_endpoint=credentials_2['IBM_AUTH_ENDPOINT'],
                    config=Config(signature_version='oauth'),
                    endpoint_url=credentials_2['ENDPOINT'])

def get_file(filename):
    '''Retrieve file from Cloud Object Storage'''
    fileobject = cos.get_object(Bucket=credentials_2['BUCKET'], Key=filename)['Body']
    return fileobject

def load_string(fileobject):
    '''Load the file contents into a Python string'''
    text = fileobject.read()
    text = text.strip(b"0xc7")
    return text.decode('latin-1')

def put_file(filename, filecontents):
    '''Write file to Cloud Object Storage'''
    resp = cos.put_object(Bucket=credentials_2['BUCKET'], Key=filename, Body=filecontents)
    return resp

In [ ]:
sampleTextFileName = "data.csv"
text = load_string(get_file(sampleTextFileName))
text = text.strip("|'|\"|,|-|_|.|""|<|>|/|.|(|)|")
now_text = text.split('\n')

In [ ]:
import nltk
tokenizer = nltk.RegexpTokenizer(r"\w+")
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, CategoriesOptions,KeywordsOptions,SentimentOptions
import pandas as pd

authenticator = IAMAuthenticator('Sna9bingjNn5_WehY2FD3VNeRQGUjnPIDTqY4t6NpB4X')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator
)
i=1
print(len(now_text))
responses=[]
lst = []
while i< len(now_text)-1:
    now_text1 = now_text[i]
    now_text1 = now_text1.strip("|'|<|>|/|.")
    i+=1
    natural_language_understanding.set_service_url('https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/a79b8129-b17d-4fa8-8703-ca4ee7d3df6e')
    try:
        s = tokenizer.tokenize(now_text1.replace("NEWLINE_TOKEN",'').replace("_"," ").replace("1"," ").replace("2"," ").replace("3"," ").replace("4"," ").replace("5"," ").replace("6"," ").replace("7"," ").replace("8"," ").replace("9"," ").replace("0"," ").lower())
        listToStr = ' '.join([str(elem) for elem in s])
        response = natural_language_understanding.analyze(
            text=listToStr,
            features=Features(sentiment=SentimentOptions(targets=[listToStr]))).get_result()
        lst.append([now_text1.split(',')[0],response.get('sentiment').get('document').get('label'),response.get('sentiment').get('document').get('score')])
        abcd = pd.DataFrame(lst, columns=['rev_id','sentiment','sentiment_score'])
    except Exception as e:
        print(e)
        continue

77974


ERROR:root:unsupported text language: unknown
Traceback (most recent call last):
  File "/home/spark/shared/user-libs/python3.6/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: unsupported text language: unknown, Code: 400 , X-global-transaction-id: 5cee8c51411891cc8073b15375e58306


Error: unsupported text language: unknown, Code: 400 , X-global-transaction-id: 5cee8c51411891cc8073b15375e58306


ERROR:root:unsupported text language: unknown
Traceback (most recent call last):
  File "/home/spark/shared/user-libs/python3.6/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: unsupported text language: unknown, Code: 400 , X-global-transaction-id: df707f6c6be6460a3cff3e51fcfde1e5


Error: unsupported text language: unknown, Code: 400 , X-global-transaction-id: df707f6c6be6460a3cff3e51fcfde1e5


...

Error: unsupported text language: aa, Code: 400 , X-global-transaction-id: d65e14d607659abbd228745ba0dbd0a0


ERROR:root:unsupported text language: micmac
Traceback (most recent call last):
  File "/home/spark/shared/user-libs/python3.6/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: unsupported text language: micmac, Code: 400 , X-global-transaction-id: 5ca83f759f97d02449fd19179c3f9d49


Error: unsupported text language: micmac, Code: 400 , X-global-transaction-id: 5ca83f759f97d02449fd19179c3f9d49


ERROR:root:unsupported text language: la
Traceback (most recent call last):
  File "/home/spark/shared/user-libs/python3.6/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: unsupported text language: la, Code: 400 , X-global-transaction-id: 0c3dc6cb63e7826a33bc4eb64e558764


Error: unsupported text language: la, Code: 400 , X-global-transaction-id: 0c3dc6cb63e7826a33bc4eb64e558764


In [ ]:
abcd.head()